In [ ]:
%pip install opendatasets

## Download and prepare the data

Download the dataset and prepare it for use with a SQL database.


**Reasoning**:
Download the dataset from Kaggle using the opendatasets library and examine the downloaded files.



In [ ]:
import opendatasets as od
import os

dataset_url = 'https://www.kaggle.com/datasets/lykin22/tiktok-trending-data'
od.download(dataset_url)

# List the downloaded files
dataset_name = dataset_url.split('/')[-1]
download_path = f'./{dataset_name}'
print(f"Downloaded dataset to: {download_path}")
print("Files in the dataset:")
for root, dirs, files in os.walk(download_path):
    for file in files:
        print(os.path.join(root, file))

**Reasoning**:
The dataset contains JSON files. To prepare for a SQL database, I will read each JSON file into a pandas DataFrame and then save them as CSV files, which are generally easier to import into SQL databases.



In [ ]:
json_files = [f for f in os.listdir(download_path) if f.endswith('.json')]

if json_files:
    first_json_file = os.path.join(download_path, json_files[0])
    print(f"Inspecting the first few lines of {first_json_file}:")
    with open(first_json_file, 'r', encoding='utf-8') as f:
        # Read and print the first 10 lines to get an idea of the structure
        for i in range(10):
            try:
                line = next(f)
                print(line.strip())
            except StopIteration:
                break
else:
    print("No JSON files found in the download directory.")

**Reasoning**:
Iterate through the keys of the loaded JSON data, convert each list into a DataFrame, and save each DataFrame as a CSV file.



In [ ]:
output_csv_path = './tiktok_csv_data'
os.makedirs(output_csv_path, exist_ok=True)

json_files = [f for f in os.listdir(download_path) if f.endswith('.json')]

for json_file in json_files:
    json_filepath = os.path.join(download_path, json_file)
    print(f"Processing {json_filepath}...")

    try:
        with open(json_filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)

        for key, value in data.items():
            if isinstance(value, list):
                df = pd.DataFrame(value)
                csv_filename = f"{os.path.splitext(json_file)[0]}_{key}.csv"
                csv_filepath = os.path.join(output_csv_path, csv_filename)
                df.to_csv(csv_filepath, index=False)
                print(f"Saved {csv_filepath}")
            else:
                print(f"Skipping key '{key}' in {json_file} as it is not a list.")

    except Exception as e:
        print(f"Error processing {json_file}: {e}")

print(f"CSV files saved to {output_csv_path}")

## Set up a database environment

Set up a suitable environment to run SQL queries, like using a file-based SQL database (e.g., SQLite).


**Reasoning**:
Import the sqlite3 library and create a connection to a new SQLite database file, then create a cursor object.



In [ ]:
import sqlite3

# Create a connection to the database
conn = sqlite3.connect('tiktok_trending.db')

# Create a cursor object
cursor = conn.cursor()

## Load the data into the database

### Subtask:
Load the prepared data into tables within the database.


**Reasoning**:
Iterate through the CSV files, read them into pandas DataFrames, and load them into the SQLite database as tables.



In [ ]:
import glob
import os
import pandas as pd

csv_files = glob.glob('./tiktok_csv_data/*.csv')

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    table_name = os.path.splitext(os.path.basename(csv_file))[0]
    print(f"Loading {csv_file} into table {table_name}...")
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    print(f"Successfully loaded {table_name}")


## Perform data preprocessing with sql

### Subtask:
Use SQL queries to perform basic data cleaning and preprocessing steps.


**Reasoning**:
Inspect the schema of the tables in the database and identify columns for cleaning.



In [ ]:
cursor.execute("") # 1. Insert query here...
tables = cursor.fetchall()
print("Tables in the database:")
for table in tables:
    table_name = table[0]
    print(f"\nSchema for table: {table_name}")
    cursor.execute(f"PRAGMA table_info({table_name});")
    schema = cursor.fetchall()
    for col in schema:
        print(col)

Tables in the database:


**Reasoning**:
Check for missing values in columns of interest, particularly in the 'bio' and 'stats' columns in user tables, and 'description' and 'stats' in hashtag tables.



In [ ]:
print("Checking for missing values in t_user table:")
cursor.execute("SELECT COUNT(*) FROM t_user WHERE bio IS NULL OR bio = '';")
missing_bio_t_user = cursor.fetchone()[0]
print(f"Missing or empty 'bio' in t_user: {missing_bio_t_user}")

cursor.execute("") # 2. Insert query here...
missing_stats_t_user = cursor.fetchone()[0]
print(f"Missing or empty 'stats' in t_user: {missing_stats_t_user}")

print("\nChecking for missing values in m_hashtag table:")
cursor.execute("") # 3. Insert query here...
missing_desc_m_hashtag = cursor.fetchone()[0]
print(f"Missing or empty 'description' in m_hashtag: {missing_desc_m_hashtag}")

cursor.execute("SELECT COUNT(*) FROM m_hashtag WHERE stats IS NULL OR stats = '';")
missing_stats_m_hashtag = cursor.fetchone()[0]
print(f"Missing or empty 'stats' in m_hashtag: {missing_stats_m_hashtag}")

print("\nChecking for missing values in m_music table:")
cursor.execute("") # 4. Insert query here...
missing_music_info_m_music = cursor.fetchone()[0]
print(f"Missing or empty 'musicInfo' in m_music: {missing_music_info_m_music}")

cursor.execute("SELECT COUNT(*) FROM m_music WHERE stats IS NULL OR stats = '';")
missing_stats_m_music = cursor.fetchone()[0]
print(f"Missing or empty 'stats' in m_music: {missing_stats_m_music}")

**Reasoning**:
Extract data from the nested 'stats' column in the t_user table by treating the string as JSON and creating new columns for 'followers' and 'likes'.



In [ ]:
# Corrected UPDATE statement to extract followers and likes from the 'stats' column
# This version handles single quotes around the keys
cursor.execute("""
UPDATE t_user
SET
    followers = CASE
        WHEN stats LIKE '%''followers'':%' THEN
            CAST(SUBSTR(stats,
                        INSTR(stats, '''followers'':') + LENGTH('''followers'':'),
                        INSTR(SUBSTR(stats, INSTR(stats, '''followers'':') + LENGTH('''followers'':')), ',') - 1) AS INTEGER)
        ELSE NULL
    END,
    likes = CASE
        WHEN stats LIKE '%''likes'':%' THEN
            CAST(SUBSTR(stats,
                        INSTR(stats, '''likes'':') + LENGTH('''likes'':'),
                        INSTR(SUBSTR(stats, INSTR(stats, '''likes'':') + LENGTH('''likes'':')), '}') - 1) AS INTEGER)
        ELSE NULL
    END
WHERE stats IS NOT NULL AND stats != '';
""")

conn.commit()

# Delete rows with missing or empty 'bio'
cursor.execute("DELETE FROM t_user WHERE bio IS NULL OR bio = '';")
conn.commit()

# Verify the update and deletion
print("\nVerifying extraction of followers and likes and deletion of rows with missing bio from t_user:")
cursor.execute("SELECT userId, stats, followers, likes, bio FROM t_user LIMIT 5;")
results = cursor.fetchall()
for row in results:
    print(row)

print("\nChecking for missing values in t_user table after cleaning:")
cursor.execute("SELECT COUNT(*) FROM t_user WHERE bio IS NULL OR bio = '';")
missing_bio_t_user_after = cursor.fetchone()[0]
print(f"Missing or empty 'bio' in t_user after cleaning: {missing_bio_t_user_after}")

cursor.execute("SELECT COUNT(*) FROM t_user WHERE stats IS NULL OR stats = '';")
missing_stats_t_user_after = cursor.fetchone()[0]
print(f"Missing or empty 'stats' in t_user after cleaning: {missing_stats_t_user_after}")

## Perform exploratory data analysis with sql

### Subtask:
Use SQL queries to explore the data, calculate statistics, and identify patterns.


**Reasoning**:
Execute SQL queries to calculate and display the average number of followers and likes for users in the t_user table.



In [ ]:
print("Average followers and likes for users:")
cursor.execute("") # 5. Insert query here...
avg_stats = cursor.fetchone()
print(f"Average Followers: {avg_stats[0]:,.2f}")
print(f"Average Likes: {avg_stats[1]:,.2f}")

**Reasoning**:
Execute a SQL query to find and display the top 10 users with the most followers from the `t_user` table.



In [ ]:
print("\nTop 10 users by followers:")
cursor.execute("SELECT name, followers FROM t_user ORDER BY followers DESC LIMIT 10;")
top_followers = cursor.fetchall()
for user in top_followers:
    print(f"User: {user[0]}, Followers: {user[1]:,}")

**Reasoning**:
Execute a SQL query to find and display the top 10 users with the most likes from the `t_user` table, count the number of verified and unverified users, and calculate the average followers and likes for verified vs. unverified users.



In [ ]:
print("\nTop 10 users by likes:")
cursor.execute("") # 6. Insert query here...
top_likes = cursor.fetchall()
for user in top_likes:
    print(f"User: {user[0]}, Likes: {user[1]:,}")

print("\nNumber of verified and unverified users:")
cursor.execute("SELECT verified, COUNT(*) FROM t_user GROUP BY verified;")
verification_counts = cursor.fetchall()
for count in verification_counts:
    status = "Verified" if count[0] == 1 else "Unverified"
    print(f"{status}: {count[1]}")

print("\nAverage followers and likes for verified vs. unverified users:")
cursor.execute("") # 7. Insert query here...
avg_stats_by_verification = cursor.fetchall()
for stats in avg_stats_by_verification:
    status = "Verified" if stats[0] == 1 else "Unverified"
    print(f"{status} - Average Followers: {stats[1]:,.2f}, Average Likes: {stats[2]:,.2f}")

## Summary:

### Data Analysis Key Findings

*   The dataset, initially in nested JSON format, was successfully converted into multiple flat CSV files (e.g., `t_user.csv`, `m_hashtag.csv`).
*   These CSV files were successfully loaded into separate tables in a SQLite database.
*   In the `t_user` table, 2 rows were found to have missing or empty 'bio' values. There were no missing values in the 'stats' column of `t_user`, or in the 'description', 'stats' (m\_hashtag), 'musicInfo', or 'stats' (m\_music) columns.
*   Numerical data (followers, likes) was successfully extracted from the 'stats' string column in the `t_user` table using SQL string manipulation after correcting for single quotes around the keys.
*   Rows with missing or empty 'bio' in the `t_user` table were successfully removed.
*   The average number of followers for users in the `t_user` table is approximately 23.64 million, while the average number of likes is approximately 812.28 million.
*   The top 10 users by followers and likes were identified, showing high engagement metrics for leading accounts.
*   There are significantly more verified users (22) than unverified users (5) in the `t_user` table.
*   Verified users, on average, have considerably more followers (approx. 26.8 million) and likes (approx. 928.45 million) compared to unverified users (approx. 9.74 million followers and 301.14 million likes).

### Insights or Next Steps

*   The substantial difference in average followers and likes between verified and unverified users suggests that verification status on TikTok might be strongly correlated with user popularity and engagement metrics.
*   Further analysis could involve joining tables (e.g., users with their music or hashtags) to explore relationships between content, users, and popularity, or examining trends over time if temporal data were available.


### Take Home Task

Find another TikTok dataset from Kaggle (or scraping if you're more advanced) and use the similar SQL queries and the same EDA process to analyze an interesting trend in the dataset. Use aggregate statements, joins, and filtering techniques in your queries for more practice.

## Answers
1. "SELECT name FROM sqlite_master WHERE type='table';"
2. "SELECT COUNT(*) FROM t_user WHERE stats IS NULL OR stats = '';"
3. "SELECT COUNT(*) FROM m_hashtag WHERE description IS NULL OR description = '';"
4. "SELECT COUNT(*) FROM m_music WHERE musicInfo IS NULL OR musicInfo = '';"
5. "SELECT AVG(followers), AVG(likes) FROM t_user;"
6. "SELECT name, likes FROM t_user ORDER BY likes DESC LIMIT 10;"
7. "SELECT verified, AVG(followers), AVG(likes) FROM t_user GROUP BY verified;"